全局函数

In [ ]:
from pprint import pprint

def to_time(value, ratio = 1):
    return 60 / value * ratio

to_bpm = to_time

类

In [ ]:
class SonolusServer():
    pass

客户端

In [ ]:
import urllib
import urllib.request
import urllib.error
import urllib.parse

# pprint(urllib.parse.urlparse('https://servers.sonolus.com/pjsekai/sonolus/info?localization=zhs'))



服务端

In [ ]:
import http
import http.server

class toolbox_handler(http.server.BaseHTTPRequestHandler):
    def do_GET(self):
        pprint(self.path)
        pprint(self.headers)
        pprint(self.command)
        self.headers.replace_header('Host', 'servers.sonolus.com')

        r = urllib.request.Request(url = 'https://servers.sonolus.com' + self.path, headers = self.headers, method = self.command)

        with urllib.request.urlopen(url = r) as result:
            pprint(result)
    def do_POST(self):
        pprint(self.path)
        pprint(self.headers)
        pprint(self.command)

主函数

In [ ]:
port = 9639

toolbox_server = http.server.ThreadingHTTPServer(('', port), toolbox_handler)

from google.colab.output import eval_js
print(eval_js( "google.colab.kernel.proxyPort(" + str(port) + ")" ))
toolbox_server.serve_forever()

测试

In [ ]:
import unittest